## Simulation (Planck test) 

Aim: build a Planck TOD from a Planck input map $x^{Planck}_{I00}$(generated by PySM), then use Map-Making and compare the recovered map with the Planck input convolved with QUBIC resolution, i.e., 

Posibilities: 

#### · Monochrimatic instrument (${\rm nfsub} = 1$) and no spectroimaging (${\rm N} = 1$), $\lambda_{i} = \lambda_{k} = \lambda_{p} = \lambda$

${\rm H}_{\lambda}^{-1} {\rm H}_{\lambda} C_{\lambda}^{Qubic} C_{\lambda_p}^{Planck} X^{I00} =C_{\lambda}^{Qubic} C_{\lambda_p}^{Planck} X^{I00} $
    

#### · Polychromatic instrument (${\rm nfsub} > 1$) and spectroimaging (${\rm N} > 1$)
   ${\rm H}^{-1}_{\lambda_{k}} \sum_{\lambda_i} {\rm H}_{\lambda_{i}} C_{\lambda_i}^{Qubic} \{\rm C_{\lambda_p}^{Planck} X^{I00} \} \overset{?}{=} C_{\lambda_k}^{Qubic} C_{\lambda_p}^{Planck} X^{\rm I00}$


In [ ]:
# Load modules
from __future__ import division
import sys
import os
import time
import pysm
import qubic 
import glob

import healpy as hp
import numpy as np
import matplotlib.pyplot as mp
import SpectroImLib as si

from pysimulators import FitsArray
from pysm.nominal import models
from scipy.constants import c
mp.rc('text', usetex = False)

### Using PySM (monochromatic instrument):

We use PySM to generate the Planck input map (@Planck resolution) and Qubic input map (infinite resolution).

In [ ]:
### Instrument ###
d = qubic.qubicdict.qubicDict()
dp = qubic.qubicdict.qubicDict()
d.read_from_file("parameters.dict")
d['MultiBand'] = False # no spectro imager
d['nf_sub'] = 1
dp.read_from_file("parameters.dict")
dp['MultiBand'] = False
dp['nf_sub'] = 1


### Sky ###
sky_config = {
    'synchrotron': models('s1', d['nside']),
    'dust': models('d1', d['nside']),
    'freefree': models('f1', d['nside']), #not polarized
    'cmb': models('c1', d['nside']),
    'ame': models('a1', d['nside'])} #not polarized



In [ ]:
planck_sky = si.Planck_sky(sky_config, d)
x0_planck = planck_sky.get_sky_map()

In [ ]:
p = qubic.get_pointing(d)

### Put Q and U to zero ###
x0_planckI = np.copy(x0_planck)
x0_planckI[..., 1:3] = 0

### Planck TOD ###
TODp, map_planck_conv = si.create_TOD(dp, p, x0_planckI)

In [ ]:
#Numbers of subbands for spectroimaging
noutmin = 1
noutmax = 1

#recons for Planck
for nf_sub_rec in np.arange(noutmin, noutmax+1):
    print 'recons in {} subbands'.format(nf_sub_rec)
    maps_reconpI, cov, nus, nus_edge, planck_conv_nusrec = si.reconstruct_maps(
        TODp, d, p, nf_sub_rec, x0 = x0_planckI)
    
    cov = np.sum(cov, axis=0)
    maxcov = np.max(cov)
    unseen = cov < maxcov*0.3
    maps_reconpI[unseen,:] = hp.UNSEEN


In [ ]:
###Map for  I != 0
mp.clf()
mp.figure(figsize=(15,15))
j=1
#inrec = 0
nrow = 3
ncolumn =3
size = 300
carte = np.empty((3,size,size))
for istokes in [0,1,2]:

    if istokes == 0:
        minx = -205.
        maxx = 152
    elif istokes == 1:
        minx = -2.72
        maxx = 2.71
    elif istokes == 2:
        minx = -3.08
        maxx = 3.27
    hp.gnomview(maps_reconpI[:,istokes], sub = (nrow,ncolumn,j), #min=minx, max=maxx,
        rot = qubic.equ2gal(d['RA_center'], d['DEC_center']), xsize = size, reso = 5, 
        title = r'Rec. Planck')

    hp.gnomview(map_planck_conv[:,istokes], sub = (nrow,ncolumn,j+1), #min=minx, max=maxx,
        rot = qubic.equ2gal(d['RA_center'], d['DEC_center']), xsize = size, reso = 5, 
        title = r'Planck convolved (with reconstruct_map)  {}'.format(d['kind'][istokes]))

    carte[istokes,:,:] = hp.gnomview(maps_reconpI[:,istokes]-planck_conv_nusrec[:,istokes], sub = (nrow,ncolumn,j+2), #min=minx, max=maxx,
        rot = qubic.equ2gal(d['RA_center'], d['DEC_center']), xsize = size, reso = 5, 
        title = r'Rec. Planck-Planck convolved {}'.format(d['kind'][istokes]), return_projected_map=True)
    
    j += ncolumn
    
mp.show()
    

In [ ]:
print(carte.shape)
print(np.mean(carte[0]), np.std(carte[0]))

In [ ]:
hp.mollview(maps_reconpI[:,0]-map_planck_conv[:,0],# sub = (nrow,ncolumn,j+2), #min=minx, max=maxx,
        rot = qubic.equ2gal(d['RA_center'], d['DEC_center']), min=-1, max=1,#xsize = size, reso = 5, 
        title = r'Rec. Planck-Planck convolved {}'.format(d['kind'][0]))
mp.show()